# Required Capstone Component 12.1

In [24]:
# Import any necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, ConstantKernel as C, Matern
from scipy.stats import norm


# load initial inputs
x_1 = np.load("initial_data/function_1/initial_inputs.npy")
x_2 = np.load("initial_data/function_2/initial_inputs.npy")
x_3 = np.load("initial_data/function_3/initial_inputs.npy")
x_4 = np.load("initial_data/function_4/initial_inputs.npy")
x_5 = np.load("initial_data/function_5/initial_inputs.npy")
x_6 = np.load("initial_data/function_6/initial_inputs.npy")
x_7 = np.load("initial_data/function_7/initial_inputs.npy")
x_8 = np.load("initial_data/function_8/initial_inputs.npy")

# load initial outputs
y_1 = np.load("initial_data/function_1/initial_outputs.npy")
y_2 = np.load("initial_data/function_2/initial_outputs.npy")
y_3 = np.load("initial_data/function_3/initial_outputs.npy")
y_4 = np.load("initial_data/function_4/initial_outputs.npy")
y_5 = np.load("initial_data/function_5/initial_outputs.npy")
y_6 = np.load("initial_data/function_6/initial_outputs.npy")
y_7 = np.load("initial_data/function_7/initial_outputs.npy")
y_8 = np.load("initial_data/function_8/initial_outputs.npy")

## Function 1:

A function with a 2D input and 1D output. 

*Application: Detect likely contamination sources in a two-dimensional area, such as a radiation field, where only proximity yields a non-zero reading. The system uses Bayesian optimisation to tune detection parameters and reliably identify both strong and weak sources.*

I will use a Matern kernel because it is a generalisation of RBF and allows you to control for smoothness; and because the application is spatial, and the Matern kernel originated from spatial statistics. The Matern function has two parameters: length scale and $\nu$. Length scale is the correlation decay parameter, and $\nu$ controls the smoothness. 

The larger the length scale, the more correlated far away points are, while smaller values of length scales means the correlation between points decays faster with distance. Because I will not be doing much tuning of the parameters of my kernel, I will use the average distance between points as my length scale. 

$\nu = 1.5$ corresponds to once differentiable functions, and $\nu = 2.5$ corresponds to twice differentiable functions. As $\nu$ tends towards infinity, the kernel becomes equivalent to the RBF kernel. So, the smaller $\nu$, the less smooth the function.

Source: https://scikit-learn.org/stable/modules/generated/sklearn.gaussian_process.kernels.Matern.html

In [ ]:
#look at the data
print(x_1)
print(y_1)

# look at average distance between points
n = len(x_1[0])
out_1 = np.sum(x_1[0] * np.arange(n-1, -n, -2) ) / (n*(n-1) / 2)
out_2= np.sum(x_1[1] * np.arange(n-1, -n, -2) ) / (n*(n-1) / 2)
print(abs(out_1))
print(abs(out_2))

[[0.31940389 0.76295937]
 [0.57432921 0.8798981 ]
 [0.73102363 0.73299988]
 [0.84035342 0.26473161]
 [0.65011406 0.68152635]
 [0.41043714 0.1475543 ]
 [0.31269116 0.07872278]
 [0.68341817 0.86105746]
 [0.08250725 0.40348751]
 [0.88388983 0.58225397]]
[ 1.32267704e-079  1.03307824e-046  7.71087511e-016  3.34177101e-124
 -3.60606264e-003 -2.15924904e-054 -2.08909327e-091  2.53500115e-040
  3.60677119e-081  6.22985647e-048]
0.4435554854300381
0.30556889047452995


In [53]:
# GP assumption
noise_assumption = 1e-10

# kernel parameters
length_scale = [0.1, 0.1]
nu = 1.5

# set up evaluation grid:
x_11 = np.linspace(0, 1, 100)
x_12 = np.linspace(0, 1, 100)
x_11, x_12 = np.meshgrid(x_11, x_12)
x_1_grid = np.column_stack([x_11.ravel(), x_12.ravel()])

# Define and fit GP
kernel = Matern(length_scale = length_scale, nu = nu, length_scale_bounds=(1e-2, 1e5))
model = GaussianProcessRegressor(kernel = kernel, alpha = noise_assumption)
model.fit(np.array(x_1), np.array(y_1))

# Get the predicted (posterior) mean and standard deviation for each grid point
post_mean, post_std = model.predict(x_1_grid, return_std=True)
print(post_mean)
print(post_std)

# Calculate the UCB aquisition function
kappa = 0.0001
UCB = post_mean + kappa * post_std

# Get the next query point
max_idx_1 = np.argmax(UCB)  
next_point_1 = x_1_grid[max_idx_1] 
print(next_point_1)



[7.85415802e-05 4.97632277e-05 2.30928503e-05 ... 5.28156602e-03
 5.33857113e-03 5.39323867e-03]
[0.00631399 0.00607142 0.00583362 ... 0.00645337 0.00668342 0.00691831]
[0. 1.]


## Function 2:

A function with a 2D input and 1D output. 

*Application: Imagine a black box, or a mystery ML model, that takes two numbers as input and returns a log-likelihood score. Your goal is to maximise that score, but each output is noisy, and depending on where you start, you might get stuck in a local optimum.* 

*To tackle this, you use Bayesian optimisation, which selects the next inputs based on what it has learned so far. It balances exploration with exploitation, making it well suited to noisy outputs and complex functions with many local peaks.*



## Function 3:

A function with a 3D input and 1D output. 

*Application: You’re working on a drug discovery project, testing combinations of three compounds to create a new medicine.*

*Each experiment is stored in initial_inputs.npy as a 3D array, where each row lists the amounts of the three compounds used. After each experiment, you record the number of adverse reactions, stored in initial_outputs.npy as a 1D array.*

*Your goal is to minimise side effects; in this competition, it is framed as maximisation by optimising a transformed output (e.g. the negative of side effects).*

## Function 4:

A function with a 4D input and a 1D output.

*Application: Address the challenge of optimally placing products across warehouses for a business with high online sales, where accurate calculations are costly and only feasible biweekly. To speed up decision-making, an ML model approximates these results within hours. The model has four hyperparameters to tune, and its output reflects the difference from the expensive baseline. Because the system is dynamic and full of local optima, it requires careful tuning and robust validation to find reliable, near-optimal solutions.*

## Function 5:

A function with a 4D input and a 1D output.

*Application: You’re tasked with optimising a four-variable black-box function that represents the yield of a chemical process in a factory. The function is typically unimodal, with a single peak where yield is maximised.*

*Your goal is to find the optimal combination of chemical inputs that delivers the highest possible yield, using systematic exploration and optimisation methods.*

## Function 6:

A function with a 5D input and a 1D output.

*Application: You’re optimising a cake recipe using a black-box function with five ingredient inputs, for example flour, sugar, eggs, butter and milk. Each recipe is evaluated with a combined score based on flavour, consistency, calories, waste and cost, where each factor contributes negative points as judged by an expert taster. This means the total score is negative by design.*

*To frame this as a maximisation problem, your goal is to bring that score as close to zero as possible or, equivalently, to maximise the negative of the total sum.*

## Function 7:

A function with a 6D input and a 1D output.

*Application: You’re tasked with optimising an ML model by tuning six hyperparameters, for example learning rate, regularisation strength or number of hidden layers. The function you’re maximising is the model’s performance score (such as accuracy or F1), but since the relationship between inputs and output isn’t known, it’s treated as a black-box function.*

*Because this is a commonly used model, you might benefit from researching best practices or literature to guide your initial search space. Your goal is to find the combination of hyperparameters that yields the highest possible performance.*

## Function 8:

A function with a 8D input and a 1D output.

*Application: You’re optimising an eight-dimensional black-box function, where each of the eight input parameters affects the output, but the internal mechanics are unknown.*

*Your objective is to find the parameter combination that maximises the function’s output, such as performance, efficiency or validation accuracy. Because the function is high-dimensional and likely complex, global optimisation is hard, so identifying strong local maxima is often a practical strategy.*

*For example, imagine you’re tuning an ML model with eight hyperparameters: learning rate, batch size, number of layers, dropout rate, regularisation strength, activation function (numerically encoded), optimiser type (encoded) and initial weight range. Each input set returns a single validation accuracy score between 0 and 1. Your goal is to maximise this score.*